### Match and questionnaire data from speed dating experiment run by Columbia professors Ray Fisman and Sheena Iyengar

https://data.world/annavmontoya/speed-dating-experiment

data is awful. likely corrupted by record keeper. do not trust.

In [109]:
dfo = pd.read_csv('Speed Dating Data.csv', encoding='latin-1')
dfo.shape

(8378, 195)

In [110]:
import json
dk = json.load(open('key.json'))
dk = {k: {int(kk): vv for kk, vv in v.items()} for k, v in dk.items()}

In [111]:
c = list(dk.keys())

In [112]:
for k, v in dk.items():
    if len(v) > 0:
        print('mapping {}'.format(k))
        df[k] = dfo[k].map(lambda x: v[x] if not np.isnan(x) else x)
df = df[c]

mapping condtn
mapping race
mapping field_cd
mapping go_out
mapping gender
mapping career_c
mapping date
mapping goal
mapping samerace


In [115]:
df = df.dropna(subset=['iid', 'pid'], how='any').copy()
for k in ['iid', 'pid']:
    df[k] = df[k].astype(int)

In [116]:
for k in ['idg', 'iid', 'round', 'partner', 'position']:
    df[k] = df[k].astype('category')
for k in ['order']:
    df[k] = df[k].astype('float')

In [119]:
for k in df:
    if df[k].dtype.name == 'object':
        df[k] = df[k].astype('category')
    if df[k].dtype.name.startswith('int'):
        print(k, df[k].min(), df[k].max())

match 0 1
pid 1 552
dec_o 0 1


In [120]:
df.shape

(8368, 53)

In [121]:
df.head(n=1).T

,0
dining,9
idg,1
field,Law
pf_o_att,35
condtn,limited choice
movies,10
exphappy,3
mn_sat,NaN
gaming,1
iid,1


In [142]:
# use iid
a = df.set_index(['iid', 'pid'])[['match', 'dec_o']]
b = a.reset_index()
x = b['iid'].copy()
b['iid'] = b['pid'].copy()
b['pid'] = x
b = b.set_index(['iid', 'pid'])['dec_o']
a = a['match']
a = pd.concat([a, b], axis=1)

In [144]:
df.age_o

0       27.0
1       22.0
2       22.0
3       23.0
4       24.0
5       25.0
6       30.0
7       27.0
8       28.0
9       24.0
10      27.0
11      22.0
12      22.0
13      23.0
14      24.0
15      25.0
16      30.0
17      27.0
18      28.0
19      24.0
20      27.0
21      22.0
22      22.0
23      23.0
24      24.0
25      25.0
26      30.0
27      27.0
28      28.0
29      24.0
        ... 
8348    27.0
8349    25.0
8350    24.0
8351    26.0
8352    24.0
8353    29.0
8354    22.0
8355    22.0
8356    28.0
8357    26.0
8358    23.0
8359     NaN
8360    23.0
8361    23.0
8362    23.0
8363    27.0
8364    30.0
8365    23.0
8366    26.0
8367    28.0
8368    26.0
8369    22.0
8370    27.0
8371    25.0
8372    24.0
8373    26.0
8374    24.0
8375    29.0
8376    22.0
8377    22.0
Name: age_o, dtype: float64